## Installing Packages ##

In [36]:
!pip install nfl_data_py

  Using cached pandas-1.5.3-cp312-cp312-linux_x86_64.whl
  Attempting uninstall: pandas
    Found existing installation: pandas 2.3.2
    Uninstalling pandas-2.3.2:
      Successfully uninstalled pandas-2.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ortools 9.14.6206 requires pandas>=2.0.0, but you have pandas 1.5.3 which is incompatible.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 1.5.3 which is incompatible.
xarray 2025.8.0 requires pandas>=2.2, but you have pandas 1.5.3 which is incompatible.
cudf-cu12 25.6.0 requires pandas<2.2.4dev0,>=2.0, but you have pandas 1.5.3 which is incompatible.
geopandas 1.1.1 requires pandas>=2.0.0, but you have pandas 1.5.3 which is incompatible.
mizani 0.13.5 requires pandas>=2.2.0, but you have pandas 1.5.3 which is incompatible.
dask-cudf-cu12 25.6.0 requires pandas<2.2.4dev0,>=2.0, but you have pan

In [37]:
!pip install xgboost

In [38]:
!pip install ortools

  Using cached pandas-2.3.2-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (91 kB)
Using cached pandas-2.3.2-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (12.0 MB)
  Attempting uninstall: pandas
    Found existing installation: pandas 1.5.3
    Uninstalling pandas-1.5.3:
      Successfully uninstalled pandas-1.5.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
nfl-data-py 0.3.3 requires pandas<2.0,>=1.0, but you have pandas 2.3.2 which is incompatible.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.3.2 which is incompatible.
cudf-cu12 25.6.0 requires pandas<2.2.4dev0,>=2.0, but you have pandas 2.3.2 which is incompatible.
dask-cudf-cu12 25.6.0 requires pandas<2.2.4dev0,>=2.0, but you have pandas 2.3.2 which is incompatible.


In [39]:
!pip install scikit-learn

In [40]:
!pip install pandas

In [59]:
!pip -q install rapidfuzz


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 21.1 MB/s eta 0:00:00


Packages

In [41]:
import pandas as pd, numpy as np
from nfl_data_py import import_weekly_data, import_pbp_data,import_schedules
from sklearn.linear_model import RidgeCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import xgboost as xgb
import re

League Object

In [42]:
LEAGUE = {
    "teams": 12,
    "roster": {"QB":1, "RB":2, "WR":2, "TE":1, "WRT":1, "DEF":1, "K":1, "BENCH":6},
    "ir_slots": 2,
    "scoring": {
        # Passing
        "pass_yd": 0.04,   # 1 pt per 25 yards
        "pass_td": 4,
        "int": -1,

        # Rushing
        "rush_yd": 0.1,    # 1 pt per 10 yards
        "rush_td": 6,

        # Receiving
        "rec_yd": 0.1,     # 1 pt per 10 yards
        "rec_td": 6,
        "rec": 0.5,        # half-PPR (adjust if full/none)

        # Kicker (K)
        "fgm_0_39": 3,     # made FG < 40 yards
        "fgm_40_49": 4,
        "fgm_50_plus": 5,
        "fg_miss": -1,     # missed FG (sometimes only for 0–39 yds, check your league)
        "pat_made": 1,
        "pat_miss": -1,

        # Defense / Special Teams (DEF)
        "def_td": 6,       # DEF/ST TD (pick-six, punt return, etc.)
        "def_int": 2,      # interception
        "def_fum_rec": 2,  # fumble recovery
        "def_sack": 1,
        "def_safety": 2,
        "def_block": 2,    # blocked kick
        "def_ret_td": 6,   # kick/punt return TD (sometimes same as def_td)

        "pa_0": 10,        # shutout
        "pa_1_6": 7,
        "pa_7_13": 4,
        "pa_14_20": 1,
        "pa_21_27": 0,
        "pa_28_34": -1,
        "pa_35_plus": -4
    }
}


FLEX_RULES = {
    "WRT": {"WR","RB","TE"},          # Yahoo W/R/T flex
}

## Data Cleaning


In [43]:
adp = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/FantasyPros_2025_Overall_ADP_Rankings.csv")
# Ensure it has at least: player_name, position, adp (float)
print("ADP Head",adp.columns.values.tolist())
print("ADP",adp.head(1))

ADP Head ['Rank', 'Player', 'Team', 'Bye', 'POS', 'Yahoo', 'Sleeper', 'RTSports', 'AVG', 'Real-Time (?)']
ADP    Rank         Player Team   Bye  POS  Yahoo  Sleeper  RTSports  AVG  \
0   1.0  Ja'Marr Chase  CIN  10.0  WR1    1.0      1.0       1.0  1.0   

  Real-Time (?)  
0             1  


In [44]:
# Clean ADP
adp = adp.rename(columns={"Player":"player_name","POS":"position","Yahoo":"adp_rank"}).copy()
adp["position"] = adp["position"].astype(str).str.extract(r'([A-Z]+)')
adp = adp[["player_name","position","adp_rank"]].dropna(subset=["player_name","position"])
adp["adp_rank"] = pd.to_numeric(adp["adp_rank"], errors="coerce")



In [45]:
def starting_slots(league=LEAGUE):
    r = league["roster"].copy()
    r.pop("BENCH", None)
    return r

def replacement_rank(pos, league=LEAGUE):
    """Direct starters + fair share from any flex slot that accepts this position."""
    teams = league["teams"]
    r = starting_slots(league)
    starters = teams * r.get(pos, 0)
    for slot, count in r.items():
        if slot in FLEX_RULES and pos in FLEX_RULES[slot]:
            eligible = len(FLEX_RULES[slot])
            starters += int(round(teams * count / max(eligible, 1)))
    return max(starters, 1)

def def_pa_points(pa, s):
    pa = 0 if pd.isna(pa) else int(pa)
    if pa == 0:                   return s.get("pa_0", 0)
    if 1 <= pa <= 6:              return s.get("pa_1_6", 0)
    if 7 <= pa <= 13:             return s.get("pa_7_13", 0)
    if 14 <= pa <= 20:            return s.get("pa_14_20", 0)
    if 21 <= pa <= 27:            return s.get("pa_21_27", 0)
    if 28 <= pa <= 34:            return s.get("pa_28_34", 0)
    return s.get("pa_35_plus", 0)


In [46]:
SEASONS = [2024]  # we’ll use last season as baseline; can extend to 2022–2024 for features

weekly = import_weekly_data(SEASONS)

# Basic fantasy points for skill players (QB/RB/WR/TE)
s = LEAGUE["scoring"]
def fp_skill(row):
    p = 0.0
    p += row.get("passing_yards",0)*s["pass_yd"] + row.get("passing_tds",0)*s["pass_td"] + row.get("interceptions",0)*s["int"]
    p += row.get("rushing_yards",0)*s["rush_yd"] + row.get("rushing_tds",0)*s["rush_td"]
    p += row.get("receiving_yards",0)*s["rec_yd"] + row.get("receiving_tds",0)*s["rec_td"] + row.get("receptions",0)*s["rec"]
    return p

# Kickers: map typical columns if available (nfl_data_py names may differ by season)
def fp_k(row):
    p = 0.0
    # These fields may be named differently; adjust if needed by inspecting weekly.columns
    p += row.get("field_goals_made_0_19",0)*s["fgm_0_39"]
    p += row.get("field_goals_made_20_29",0)*s["fgm_0_39"]
    p += row.get("field_goals_made_30_39",0)*s["fgm_0_39"]
    p += row.get("field_goals_made_40_49",0)*s["fgm_40_49"]
    p += row.get("field_goals_made_50_plus",0)*s["fgm_50_plus"]
    p += row.get("field_goals_missed",0)*s["fg_miss"]
    p += row.get("extra_points_made",0)*s["pat_made"]
    p += row.get("extra_points_missed",0)*s["pat_miss"]
    return p

Downcasting floats.


In [47]:
wk = weekly.copy()
wk["fp_skill"] = wk.apply(fp_skill, axis=1)
wk["fp_k"] = wk.apply(fp_k, axis=1)

agg_players = (wk.groupby(["season","player_name","position"], as_index=False)
                 .agg(games=("week","nunique"),
                      fp_skill=("fp_skill","sum"),
                      fp_k=("fp_k","sum")))

In [48]:
def pick_ppg(row):
    if row["position"]=="K":   pts = row["fp_k"]
    else:                      pts = row["fp_skill"]
    return pts / max(row["games"], 1)

agg_players["ppg"] = agg_players.apply(pick_ppg, axis=1)


In [68]:
import re, numpy as np, pandas as pd
from rapidfuzz import fuzz, process

# ---------- helpers ----------
SUFFIXES = (" jr"," sr"," ii"," iii"," iv"," v")
def norm_name(s: str) -> str:
    s = str(s).lower()
    # strip D/ST words so "Buffalo Bills D/ST" -> "buffalo bills"
    s = re.sub(r"\b(d/?st|dst|defense|def|special teams)\b", "", s)
    s = re.sub(r"[^a-z\s]", "", s)
    for suf in SUFFIXES:
        s = s.replace(suf, "")
    s = re.sub(r"\s+", " ", s).strip()
    return s

# Yahoo defensive labels to keep or drop
IDP_POS = {"D","DL","DB","LB","DT","DE","CB","S"}
DEF_ALIASES = {
  "DST":"DEF","D/ST":"DEF","DEFENSE":"DEF",
  "DEFENSE/SPECIAL TEAMS":"DEF","DEFENSE SPECIAL TEAMS":"DEF","ST":"DEF"
}
# team abbr -> full name for DEF
TEAM_ABBR_TO_NAME = {
    "ARI":"arizona cardinals","ATL":"atlanta falcons","BAL":"baltimore ravens","BUF":"buffalo bills",
    "CAR":"carolina panthers","CHI":"chicago bears","CIN":"cincinnati bengals","CLE":"cleveland browns",
    "DAL":"dallas cowboys","DEN":"denver broncos","DET":"detroit lions","GB":"green bay packers",
    "HOU":"houston texans","IND":"indianapolis colts","JAX":"jacksonville jaguars","KC":"kansas city chiefs",
    "LV":"las vegas raiders","LAC":"los angeles chargers","LAR":"los angeles rams","MIA":"miami dolphins",
    "MIN":"minnesota vikings","NE":"new england patriots","NO":"new orleans saints","NYG":"new york giants",
    "NYJ":"new york jets","PHI":"philadelphia eagles","PIT":"pittsburgh steelers","SEA":"seattle seahawks",
    "SF":"san francisco 49ers","TB":"tampa bay buccaneers","TEN":"tennessee titans","WAS":"washington commanders"
}

# ---------- 1) Projections side: normalize positions; map DEF team codes to names ----------
# Expect: projections has ["player_name","position","proj_ppg_2025"]
proj_keys = projections.copy()
proj_keys["position"] = proj_keys["position"].astype(str).str.upper().str.extract(r"([A-Z/]+)")
proj_keys["position"] = proj_keys["position"].replace(DEF_ALIASES)

# drop any IDP rows if they slipped in
proj_keys = proj_keys[~proj_keys["position"].isin(IDP_POS)]

# map DEF team abbreviations (BUF, SF, etc.) to full names for matching with ADP D/ST rows
is_def = proj_keys["position"].eq("DEF")
proj_keys.loc[is_def, "player_name"] = (
    proj_keys.loc[is_def, "player_name"].map(TEAM_ABBR_TO_NAME)
    .fillna(proj_keys.loc[is_def, "player_name"])
)

# build name_key
proj_keys["name_key"] = proj_keys["player_name"].apply(norm_name)

# ---------- 2) ADP side: standardize columns; collapse to DEF; drop IDP ----------
# If your ADP was already cleaned, start from that; else standardize here:
adp_std = adp.copy()
if "Player" in adp_std.columns: adp_std = adp_std.rename(columns={"Player":"player_name"})
if "POS" in adp_std.columns:    adp_std = adp_std.rename(columns={"POS":"position"})
if "Yahoo" in adp_std.columns:  adp_std = adp_std.rename(columns={"Yahoo":"adp_rank"})

adp_std = adp_std[["player_name","position","adp_rank"]].dropna(subset=["player_name","position"])
adp_std["position"] = adp_std["position"].astype(str).str.upper().str.extract(r"([A-Z/]+)")
adp_std["position"] = adp_std["position"].replace(DEF_ALIASES)

# keep only positions we actually draft in this league
KEEP_POS = {"QB","RB","WR","TE","K","DEF"}
adp_std = adp_std[adp_std["position"].isin(KEEP_POS)]
adp_std["adp_rank"] = pd.to_numeric(adp_std["adp_rank"], errors="coerce")
adp_std["name_key"] = adp_std["player_name"].apply(norm_name)

# if duplicates (same name_key+position) exist, keep the best (lowest ADP rank)
adp_std = (adp_std.sort_values("adp_rank", ascending=True)
                   .drop_duplicates(subset=["name_key","position"], keep="first"))

# ---------- 3) Exact merge first ----------
proj_with_adp = proj_keys.merge(
    adp_std[["name_key","position","adp_rank"]],
    on=["name_key","position"], how="left"
)

print("Exact match rate:", f"{proj_with_adp['adp_rank'].notna().mean():.1%}")

# ---------- 4) Fuzzy fill ONLY remaining NaNs (position-aware) ----------
missing_idx = proj_with_adp.index[proj_with_adp["adp_rank"].isna()].tolist()
if missing_idx:
    adp_by_pos = {p: df.reset_index(drop=True) for p, df in adp_std.groupby("position", as_index=False)}

    def composite_score(q, cand):
        return max(fuzz.WRatio(q, cand), fuzz.token_sort_ratio(q, cand), fuzz.partial_ratio(q, cand))

    for i in missing_idx:
        pos = proj_with_adp.at[i, "position"]
        qname = proj_with_adp.at[i, "player_name"]
        q = norm_name(qname)
        cand_df = adp_by_pos.get(pos)
        if cand_df is None or cand_df.empty:
            continue
        names = cand_df["player_name"].tolist()
        k = min(50, len(names))
        cands = process.extract(q, names, scorer=fuzz.WRatio, limit=k)

        best_score, best_rank = 0, np.nan
        for cand_name, wr, idx in cands:
            score = composite_score(q, norm_name(cand_name))
            if score >= 88 or score >= 70:
                rank = cand_df.at[idx, "adp_rank"]
                if pd.isna(rank):
                    continue
                if (score > best_score) or (score == best_score and rank < best_rank):
                    best_score, best_rank = score, rank

        # soft fallback: accept top candidate if close
        if pd.isna(best_rank) and cands and cands[0][1] >= 62:
            best_rank = cand_df.at[cands[0][2], "adp_rank"]

        proj_with_adp.at[i, "adp_rank"] = best_rank

print("Post-fuzzy match rate:", f"{proj_with_adp['adp_rank'].notna().mean():.1%}")

# ---------- 5) Build VORP board ----------
board_core = vorp_table(
    proj_with_adp.rename(columns={"player_name":"player_name"})[["player_name","position","proj_ppg_2025"]],
    proj_col="proj_ppg_2025"
)

right_df = (proj_with_adp[["player_name","position","adp_rank"]]
            .drop_duplicates(subset=["player_name","position"], keep="first"))

board = (board_core.merge(right_df, on=["player_name","position"], how="left")
                    .sort_values(["VORP","proj_ppg_2025","adp_rank"], ascending=[False, False, True])
                    .reset_index(drop=True))

print(board.head(12)[["player_name","position","proj_ppg_2025","VORP","adp_rank"]])


Exact match rate: 3.6%
Post-fuzzy match rate: 43.1%
   player_name position  proj_ppg_2025       VORP  adp_rank
0   B.Robinson       RB      23.433333  12.783333       2.0
1    S.Barkley       RB      21.435000  10.785000       3.0
2      J.Gibbs       RB      20.188889   9.538889       4.0
3      D.Henry       RB      19.678947   9.028947       7.0
4      J.Chase       WR      19.970588   8.470588       1.0
5     J.Taylor       RB      17.364286   6.714286      17.0
6    L.Jackson       QB      25.607368   6.616702      21.0
7       R.Rice       WR      17.633333   6.133333      64.0
8     J.Jacobs       RB      16.705556   6.055556      16.0
9     A.Kamara       RB      16.521429   5.871429      45.0
10    G.Kittle       TE      13.173333   5.714510      30.0
11     J.Mixon       RB      16.118750   5.468750      74.0


In [69]:
# Check which DEF rows in projections are still missing ADP
def_missing = proj_with_adp[
    (proj_with_adp["position"]=="DEF") & (proj_with_adp["adp_rank"].isna())
][["player_name","name_key"]]

print(f"Total DEF in projections: {sum(proj_with_adp['position']=='DEF')}")
print(f"Missing ADP for DEF: {len(def_missing)}\n")

if not def_missing.empty:
    print("Examples of missing DEF (projection side):")
    print(def_missing.head(15))

    # Also show what ADP has for DEF
    adp_defs = adp_std[adp_std["position"]=="DEF"][["player_name","name_key","adp_rank"]]
    print("\nExamples of DEF from ADP:")
    print(adp_defs.head(15))

    # Compare sets for overlap
    proj_set = set(def_missing["name_key"])
    adp_set  = set(adp_defs["name_key"])
    print("\nProjection DEF keys not in ADP:", sorted(proj_set - adp_set)[:20])
    print("ADP DEF keys not in projections:", sorted(adp_set - proj_set)[:20])


Total DEF in projections: 32
Missing ADP for DEF: 7

Examples of missing DEF (projection side):
              player_name              name_key
600       atlanta falcons       atlanta falcons
603     carolina panthers     carolina panthers
612    indianapolis colts    indianapolis colts
613  jacksonville jaguars  jacksonville jaguars
617     las vegas raiders       lasegas raiders
621    new orleans saints    new orleans saints
629      tennessee titans      tennessee titans

Examples of DEF from ADP:
              player_name              name_key  adp_rank
126   Philadelphia Eagles   philadelphia eagles      82.0
124        Denver Broncos        denver broncos      84.0
149      Baltimore Ravens      baltimore ravens      97.0
138   Pittsburgh Steelers   pittsburgh steelers     101.0
154     Minnesota Vikings       minnesotaikings     127.0
177    Kansas City Chiefs    kansas city chiefs     131.0
183         Detroit Lions         detroit lions     147.0
170        Houston Texans    

In [71]:
# Known typos in ADP
adp_fix_map = {
    "minnesotaikings": "minnesota vikings",
    "san francisco ers": "san francisco 49ers"
}
adp_std["name_key"] = adp_std["name_key"].replace(adp_fix_map)

# Add missing defenses with NaN ADP (they'll drop to bottom of board)
all_teams = [
    "arizona cardinals","atlanta falcons","baltimore ravens","buffalo bills","carolina panthers",
    "chicago bears","cincinnati bengals","cleveland browns","dallas cowboys","denver broncos",
    "detroit lions","green bay packers","houston texans","indianapolis colts","jacksonville jaguars",
    "kansas city chiefs","las vegas raiders","los angeles chargers","los angeles rams","miami dolphins",
    "minnesota vikings","new england patriots","new orleans saints","new york giants","new york jets",
    "philadelphia eagles","pittsburgh steelers","san francisco 49ers","seattle seahawks",
    "tampa bay buccaneers","tennessee titans","washington commanders"
]

# Find which team DEF are missing in ADP
existing_defs = set(adp_std.loc[adp_std["position"]=="DEF","name_key"])
missing_defs = [t for t in all_teams if norm_name(t) not in existing_defs]

# Add them with NaN ADP
rows = []
for team in missing_defs:
    rows.append({"player_name": team+" D/ST",
                 "position":"DEF",
                 "adp_rank": np.nan,
                 "name_key": norm_name(team)})
adp_std = pd.concat([adp_std, pd.DataFrame(rows)], ignore_index=True)


In [72]:
# Print all 32 defenses with ADP
defs_board = adp_std[adp_std["position"]=="DEF"].copy()

# Normalize for consistency
defs_board = defs_board[["player_name","adp_rank","name_key"]].drop_duplicates()
defs_board = defs_board.sort_values("player_name").reset_index(drop=True)

print("Total defenses in ADP table:", len(defs_board))
print(defs_board.to_string(index=False))


Total defenses in ADP table: 36
             player_name  adp_rank              name_key
       Arizona Cardinals     222.0     arizona cardinals
        Baltimore Ravens      97.0      baltimore ravens
           Buffalo Bills     185.0         buffalo bills
           Chicago Bears     189.0         chicago bears
      Cincinnati Bengals     215.0    cincinnati bengals
        Cleveland Browns     226.0      cleveland browns
          Dallas Cowboys     207.0        dallas cowboys
          Denver Broncos      84.0        denver broncos
           Detroit Lions     147.0         detroit lions
       Green Bay Packers     221.0     green bay packers
          Houston Texans     148.0        houston texans
      Indianapolis Colts       NaN    indianapolis colts
    Jacksonville Jaguars       NaN  jacksonville jaguars
      Kansas City Chiefs     131.0    kansas city chiefs
       Las Vegas Raiders       NaN       lasegas raiders
    Los Angeles Chargers       NaN  los angeles chargers

In [73]:
board_defs = board[board["position"]=="DEF"].copy()

# Keep only useful columns
board_defs = board_defs[["player_name","proj_ppg_2025","VORP","adp_rank"]].drop_duplicates()

# Sort by ADP (lowest rank = earlier pick) with NaN at bottom
board_defs = board_defs.sort_values("adp_rank", na_position="last").reset_index(drop=True)

print("Total defenses in final board:", len(board_defs))
print(board_defs.to_string(index=False))


Total defenses in final board: 32
          player_name  proj_ppg_2025      VORP  adp_rank
  philadelphia eagles       1.318622  0.684874      82.0
                   LA       0.459600 -0.174149      82.0
       denver broncos       1.479337  0.845588      84.0
     baltimore ravens       0.735916  0.102167      97.0
  pittsburgh steelers       0.854337  0.220588     101.0
    minnesota vikings       1.146003  0.512255     127.0
   kansas city chiefs       0.666837  0.033088     131.0
        detroit lions       0.937670  0.303922     147.0
       houston texans       0.814863  0.181115     148.0
        buffalo bills       0.591837 -0.041912     185.0
        chicago bears       0.545513 -0.088235     189.0
 tampa bay buccaneers       0.271003 -0.362745     195.0
       miami dolphins       0.633748  0.000000     197.0
  san francisco 49ers      -0.028016 -0.661765     200.0
      new york giants       0.016101 -0.617647     204.0
 new england patriots       0.148454 -0.485294     205